In [5]:
from datasets import load_dataset, Dataset
import pandas as pd

In [6]:
dataset = load_dataset('SirNeural/flan_v2', split = 'train', streaming=True)

data = []

for i, example in enumerate(dataset):
    if i < 100000:
        data.append(example)
    else:
        break
print(data[0])


df = pd.DataFrame(data)


print(df.head(5))  


Resolving data files:   0%|          | 0/21 [00:00<?, ?it/s]

{'inputs': 'Q: This is a list of the best-selling video games of all time. The best-selling video game to date is Tetris, a tile-matching puzzle video game originally released for the Electronika 60 in 1984 and then popularised upon its Game Boy release in 1989. The game has been ported to a wide range of platforms and sold in excess of 170 million units, including 100 million paid downloads on mobile phones and 35 million as sales for the Game Boy version. Minecraft is the only other video game to have sold over 100 million units, with 122 million units sold as of February 2017. Excluding mobile sales, the best-selling video game of all time is Wii Sports with 82.81 million units sold, while Tetris is the fourth best-selling with sales of 70 million units. The highest grossing video game of all time? A: Tetris , a tile - matching puzzle video game originally released for the Electronika 60 in 1984 and then popularised upon its Game Boy release in 1989. How to explain the answer? The r

In [14]:
dataset = Dataset.from_pandas(df)

In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-2-7b-hf")


In [16]:
instruction_tokens_count = [len(tokenizer.tokenize(example)) for example in df['targets']]

In [12]:
#instruction_tokens_count = [len(tokenizer.tokenize(example["targets"])) for example in dataset]

In [17]:
print(instruction_tokens_count)

[98, 33, 36, 33, 23, 30, 23, 41, 36, 18, 32, 26, 88, 26, 24, 33, 18, 65, 48, 27, 23, 28, 99, 28, 22, 50, 27, 27, 26, 41, 30, 29, 52, 32, 31, 25, 35, 31, 24, 27, 20, 46, 24, 21, 18, 20, 23, 25, 25, 26, 73, 23, 24, 24, 32, 22, 25, 32, 33, 155, 44, 24, 90, 26, 33, 25, 19, 24, 49, 21, 33, 21, 31, 28, 25, 35, 48, 17, 48, 28, 27, 85, 146, 34, 84, 28, 24, 25, 50, 23, 134, 27, 32, 86, 38, 99, 28, 30, 27, 161, 26, 25, 33, 35, 22, 95, 25, 27, 28, 67, 22, 35, 45, 55, 30, 106, 86, 26, 66, 28, 151, 79, 21, 32, 22, 37, 30, 41, 20, 28, 30, 23, 24, 51, 79, 31, 27, 122, 33, 25, 25, 39, 27, 18, 42, 18, 21, 36, 37, 28, 35, 32, 40, 40, 35, 28, 48, 120, 44, 23, 26, 28, 24, 62, 25, 37, 26, 27, 32, 25, 28, 35, 95, 42, 66, 104, 35, 12, 74, 19, 29, 208, 68, 23, 26, 150, 43, 16, 79, 60, 38, 17, 81, 38, 75, 33, 99, 60, 87, 18, 32, 113, 38, 58, 26, 31, 29, 67, 26, 22, 25, 27, 30, 49, 70, 94, 28, 30, 29, 31, 27, 29, 23, 46, 23, 30, 42, 51, 80, 29, 58, 44, 22, 49, 20, 38, 29, 25, 27, 56, 33, 38, 16, 170, 36, 30, 36

In [20]:
df['target_token_count'] = instruction_tokens_count

filtered_df = df[df['target_token_count'] > 100]
print("Filtered DataFrame shape:", filtered_df.shape)

Filtered DataFrame shape: (5551, 4)


In [21]:
from sentence_transformers import SentenceTransformer

import faiss
from datasets import Dataset, DatasetDict

from tqdm.autonotebook import tqdm
import numpy as np

In [22]:
import pickle
dataset1 = Dataset.from_pandas(filtered_df)

sentence_model = SentenceTransformer("thenlper/gte-large")
outputs = [example["targets"] for example in dataset1]
print("Converting the text to embeddings...")
embeddings = sentence_model.encode(outputs, show_progress_bar=True)
dimensions = embeddings.shape[1]


with open('output_embeddings.pkl', 'wb') as file:
    pickle.dump(embeddings, file)

print(f"Embeddings saved to 'output_embeddings.pkl'. Shape: {embeddings.shape}")

Converting the text to embeddings...


Batches:   0%|          | 0/174 [00:00<?, ?it/s]

Embeddings saved to 'output_embeddings.pkl'. Shape: (5551, 1024)


In [23]:
def deduplicate_dataset(dataset: Dataset, model: str, threshold: float, embeddings):

  sentence_model = SentenceTransformer(model)
  #Embed every sample every row in  dataset output column
  outputs = [example["targets"] for example in dataset]
  # Using the Embedding Model we will convert the text into embeddings
  print("Convert the text to embeddings....")
  
  dimensions = embeddings.shape[1]
  print("Dimensions of the embedding", embeddings.shape)
  #Create an index using the Faiss as our Vector Database
  index = faiss.IndexFlatIP(dimensions)
  #Normalize the Embeddings
  normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
  index.add(normalized_embeddings)
  print("Filtering out near duplicates....")
  #k=2, means atmost two vectors
  D, I = index.search(normalized_embeddings, k=2)
  #In the below list, we will add the list of samples we want to keep
  to_keep=[]
  #We will define the threshold below, if the embedding is 95% similar to other embedding then we will remove that embedding
  for i in tqdm(range(len(embeddings)), desc="Filtering"):
    #If the second closest vector (D[i,1]) has cosine similarity above the threshold
    if D[i,1] >= threshold:
      #Check if the current item or its nearest neighbor is already in the to_keep list
      nearest_neighbor = I[i,1]
      if i not in to_keep and nearest_neighbor not in to_keep:
        # If not, add the current item to the list
        to_keep.append(i)
    else:
        # If the similarity is below the threshold, always keep the current item
        to_keep.append(i)
  print("List", to_keep)
  dataset = dataset.select(to_keep)
  print(dataset.to_pandas())
  return DatasetDict({"train": dataset})

dataset1 = Dataset.from_pandas(filtered_df)

deduped_dataset = deduplicate_dataset(dataset1, "thenlper/gte-large", 0.95, embeddings)

Convert the text to embeddings....
Dimensions of the embedding (5551, 1024)
Filtering out near duplicates....


Filtering:   0%|          | 0/5551 [00:00<?, ?it/s]

List [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,

In [24]:
deduped_dataset

DatasetDict({
    train: Dataset({
        features: ['inputs', 'targets', 'task', 'target_token_count', '__index_level_0__'],
        num_rows: 4302
    })
})

In [25]:
deduped_dataset["train"].to_pandas()

,inputs,targets,task,target_token_count,__index_level_0__
0,[Question]: Tom decides to get a new floor for...,The almond croissant and the salami and cheese...,cot,155,59
1,For every black & white cookie that Elsa makes...,Let's think step by step. After keeping 15 car...,cot,146,82
2,Student A:Context: The final took place on 15 ...,OK... The relevant information to answer the a...,cot,134,90
3,Q: Heart of Darkness (1899) is a novella by Po...,Lord Roger De Halys circa 1130. How to explain...,cot,161,99
4,Q: Amanda received $50 as a gift. She plans to...,Kim got 2 candy bars a week for 16 weeks so sh...,cot,106,115
...,...,...,...,...,...
4297,[Question]John paints a giant mural that is 6m...,The number that left the camp on Saturday morn...,cot,141,98266
4298,Question: Erin is watching a TV mini series of...,Step-by-step reasoning process: After one week...,cot,232,98443
4299,Question: Talia is playing football with her f...,"On the second day, he collected 3 times the nu...",cot,175,98881
4300,[Question]Hayden works for a limousine company...,"So for 3 of the days, there are 1 + 1 = 2 peop...",cot,126,98914


In [26]:
import os
os.environ["HF_TOKEN"] = "token removed for security reasons"

In [31]:
deduped_dataset.push_to_hub("rishitchugh/flan-v2-deduped")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/352 [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rishitchugh/flan-v2-deduped/commit/f633ff043ec39b84986c9b3c996497d6ece1ed7a', commit_message='Upload dataset', commit_description='', oid='f633ff043ec39b84986c9b3c996497d6ece1ed7a', pr_url=None, pr_revision=None, pr_num=None)

In [32]:
def chat_template(example):
    example["inputs"] = f"### Instruction:\n{example['inputs']}\n\n### Response:\n"
    return example

llama_dataset = deduped_dataset.map(chat_template)

Map:   0%|          | 0/4302 [00:00<?, ? examples/s]

In [33]:
llama_dataset.push_to_hub("rishitchugh/flan-llama")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/rishitchugh/flan-llama/commit/bf3971cee8c4badbdb7e765a78ba7cd7a85b8061', commit_message='Upload dataset', commit_description='', oid='bf3971cee8c4badbdb7e765a78ba7cd7a85b8061', pr_url=None, pr_revision=None, pr_num=None)